In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import Counter
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from PIL import Image
import os
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import json
BASE_DIR = "../input/cassava-leaf-disease-classification/"

with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    map_classes = {int(k) : v for k, v in map_classes.items()}

print(json.dumps(map_classes, indent=4))

{
    "0": "Cassava Bacterial Blight (CBB)",
    "1": "Cassava Brown Streak Disease (CBSD)",
    "2": "Cassava Green Mottle (CGM)",
    "3": "Cassava Mosaic Disease (CMD)",
    "4": "Healthy"
}


In [3]:
data = pd.read_csv(BASE_DIR + 'train.csv')

train_df = data
print(train_df.head())
    
# train_df, val_df = train_test_split(data, test_size = 0.1, random_state = 42, stratify = data['label'])

         image_id  label
0  1000015157.jpg      0
1  1000201771.jpg      3
2   100042118.jpg      1
3  1000723321.jpg      1
4  1000812911.jpg      3


In [4]:
class CassavaDataset(torch.utils.data.Dataset):
    def __init__(self, df, data_path=BASE_DIR, mode="train", transform=None):
        super().__init__()
        self.transform = transform
        self.df_data = df.values
        self.data_path = data_path
        self.mode = mode
        self.data_dir = "train_images" if mode == "train" else "test_images"

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, index):
        img_name, label = self.df_data[index]
        img_path = os.path.join(self.data_path, self.data_dir, img_name)
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(img)

        return image, label, index

transforms_train = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

# sprawdzi
train_dataset = CassavaDataset(train_df, transform=transforms_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, num_workers=12)

In [5]:
class RCL(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(RCL, self).__init__()
        padding = 0 if kernel_size == 1 else 1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
        self.bn2d1 = nn.BatchNorm2d(num_features=out_channels)
        self.convr = nn.Conv2d(out_channels, out_channels, kernel_size, padding=padding)
        self.bn2dr = nn.BatchNorm2d(num_features=out_channels)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv2 = self.convr(F.relu(self.bn2d1(conv1)))
        x1 = conv1.add(conv2)
        conv3 = self.convr(F.relu(self.bn2dr(x1)))
        x1 = conv1.add(conv3)
        conv4 = self.convr(F.relu(self.bn2dr(x1)))
        x1 = conv1.add(conv4)
        x1 = F.relu(self.bn2dr(x1))
        return x1
    
class Block(nn.Module):
    def __init__(self):
        super(Block, self).__init__()
        self.rcl1 = RCL(in_channels=64, out_channels=64, kernel_size=1)
        self.rcl2 = RCL(in_channels=64, out_channels=128, kernel_size=3)
        self.avgPool2d = nn.AvgPool2d(3, stride=1, padding=1)
        self.rcl3 = RCL(in_channels=64, out_channels=64, kernel_size=1)

    def forward(self, x):
        x1 = self.rcl1(x)
        x2 = self.rcl2(x)
        x3 = self.rcl3(self.avgPool2d(x))
        return torch.cat((x1, x2, x3), 1)

In [6]:
class IRCNN(nn.Module):
    def __init__(self, dropout=0.5):
        super(IRCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.bn2d2 = nn.BatchNorm2d(num_features=32)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.bn2d3 = nn.BatchNorm2d(num_features=64)

        self.block1 = Block()
        self.conv_b1 = nn.Conv2d(in_channels=256, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2d_b1 = nn.BatchNorm2d(num_features=64)
        self.maxPool2d_b1 = nn.MaxPool2d(3, stride=1, padding=1)
        self.dropout_b1 = nn.Dropout2d(p=dropout)

        self.block2 = Block()
        self.conv_b2 = nn.Conv2d(in_channels=256, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2d_b2 = nn.BatchNorm2d(num_features=64)
        self.maxPool2d_b2 = nn.MaxPool2d(3, stride=1, padding=1)
        self.dropout_b2 = nn.Dropout2d(p=dropout)

        self.block3 = Block()
        self.conv_b3 = nn.Conv2d(in_channels=256, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2d_b3 = nn.BatchNorm2d(num_features=64)
        self.globalAvgPool = nn.AdaptiveAvgPool2d(1)
        self.dropout_b3 = nn.Dropout2d(p=dropout)

        self.dense1 = nn.Linear(64, 5)
        
    def forward(self, x):
        x1 = self.conv1(x)
        x1 = F.relu(self.bn2d2(self.conv2(x1)))
        x1 = F.relu(self.bn2d3(self.conv3(x1)))
        #
        x1 = self.block1(x1)
        x1 = self.conv_b1(x1)
        x1 = self.bn2d_b1(x1)
        x1 = self.maxPool2d_b1(x1)
        x1 = self.dropout_b1(x1)
        #
        x1 = self.block2(x1)
        x1 = self.conv_b2(x1)
        x1 = self.bn2d_b2(x1)
        x1 = self.maxPool2d_b2(x1)
        x1 = self.dropout_b2(x1)
        #
        x1 = self.block3(x1)
        x1 = self.conv_b3(x1)
        x1 = self.bn2d_b3(x1)
        x1 = self.globalAvgPool(x1)
        x1 = self.dropout_b3(x1)
        x1 = x1.view(x1.size(0), -1)
        return self.dense1(x1)

In [7]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    raise Exception("Turn on GPU")  
print(dev)

net = IRCNN()
net.to(dev)

optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.9)
criterion = nn.CrossEntropyLoss()
print(len(train_loader))

cuda:0
1338


In [8]:
# print(os.mkdir('../input/'))

In [9]:
print(dev)
best_accuracy = 0
epochs = 25
results = [] # list of tuples (epoch, loss, accuracy)
# save_path = "../input/model/" + 'v1.0'

for epoch in range(epochs):
    print(f'----------EPOCH {epoch + 1}----------')
    running_loss = 0.0
    running_loss_n = 0
    correct = 0
    total = 0
    for i, data in enumerate(train_loader):
        inputs, labels, _ = data
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        optimizer.zero_grad()
        
        # forward
        outputs = net(inputs)
        
        # gather metrics
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # backward
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        running_loss_n += 1
#         if i % 100 == 0:
#             print(f'{i}: {loss.item()}')
            
    epoch_accuracy = 100 * correct / total
    current_loss = running_loss / running_loss_n
    results.append((epoch + 1, current_loss, epoch_accuracy))
    print((f"{epoch + 1} loss: {current_loss:.3f}, "
          f"accuracy: {epoch_accuracy:.2F}%, "))
#     if best_accuracy < epoch_accuracy:
#         best_accuracy = epoch_accuracy
#         torch.save((net.state_dict(), results), save_path)
    
print(results)

cuda:0
----------EPOCH 1----------
1 loss: 1.130, accuracy: 61.33%, 
----------EPOCH 2----------
2 loss: 1.044, accuracy: 63.12%, 
----------EPOCH 3----------
3 loss: 0.993, accuracy: 64.11%, 
----------EPOCH 4----------
4 loss: 0.959, accuracy: 64.81%, 
----------EPOCH 5----------
5 loss: 0.929, accuracy: 65.47%, 
----------EPOCH 6----------
6 loss: 0.899, accuracy: 66.74%, 
----------EPOCH 7----------
7 loss: 0.861, accuracy: 67.99%, 
----------EPOCH 8----------
8 loss: 0.835, accuracy: 69.32%, 
----------EPOCH 9----------
9 loss: 0.811, accuracy: 70.38%, 
----------EPOCH 10----------
10 loss: 0.796, accuracy: 71.00%, 
----------EPOCH 11----------
11 loss: 0.781, accuracy: 71.69%, 
----------EPOCH 12----------
12 loss: 0.764, accuracy: 72.80%, 
----------EPOCH 13----------
13 loss: 0.757, accuracy: 72.78%, 
----------EPOCH 14----------
14 loss: 0.743, accuracy: 73.30%, 
----------EPOCH 15----------
15 loss: 0.731, accuracy: 74.18%, 
----------EPOCH 16----------
16 loss: 0.724, accura

Evaluate and submit

In [10]:
# save_path = "../model/" + 'v1.0'

# params, results = torch.load(save_path)
# net.load_state_dict(params)

test_df = pd.DataFrame(columns=['image_id', 'label'])
test_df['image_id'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))
# print(test_df.head())
# print(data.head())
test_dataset = CassavaDataset(test_df, transform=transforms_train, mode="test")
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, num_workers=12)

with torch.no_grad():
    for data in test_loader:
        images, _, indexes = data
        if torch.cuda.is_available():
            images = images.to(dev)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        test_df['label'][indexes.numpy()] = predicted.cpu().detach().numpy()



In [11]:
test_df.to_csv('submission.csv',index=False)